In [1]:
import requests
import random
from bs4 import BeautifulSoup
import io
import time
import pandas as pd

In [2]:
tday = 1589846400
p5yday = 1431993600
per_3m = int((tday-p5yday) / 5 / 4)
pr = []
for i in range(0,20):
    pr.append(tday-per_3m)
    tday -= per_3m
pr.insert(0,1589846400)
pr.reverse()
items = ['^GSPC', 'TWD=X', '^DJI', '^IXIC']
enc_idx = []
for i in range(len(items)):
    tables = []
    while_loop = pr[:]
    while len(while_loop)!=1:
        pr_to = while_loop.pop()
        pr_from = while_loop[-1]
        item = items[i]
        url = 'https://hk.finance.yahoo.com/quote/' + item +'/history?period1=' + str(pr_from) + '&period2=' + str(pr_to) + '&interval=1d&filter=history&frequency=1d'
        table = pd.read_html(requests.get(url, headers={'User-agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}).text)
        drop_row = table[0][table[0]['日期'] == '*收市價已按拆股調整。**經調整收市價已按股息和拆股調整。'].index
        table[0].drop(drop_row, inplace=True)
        drop_row = table[0][table[0]['開市'] == '-'].index
        table[0].drop(drop_row, inplace=True)
        table[0][items[i]] = table[0]['收市*']
        table[0] = table[0][['日期',items[i]]]
        table[0]['日期'] = table[0]['日期'].str.replace('年', '/')
        table[0]['日期'] = table[0]['日期'].str.replace('月', '/')
        table[0]['日期'] = table[0]['日期'].str.replace('日', '')
        table[0]['日期'] = pd.to_datetime(table[0]['日期'])
        tables.append(table[0])
        randaom_time = random.random() * 2
        time.sleep(randaom_time)
    df_temp = pd.concat(tables, axis=0)
    df_temp.index = range(0, len(df_temp))
    # drop_row = df_gold[df_gold['日期'] == '*收市價已按拆股調整。**經調整收市價已按股息和拆股調整。'].index
    # df_gold.drop(drop_row, inplace=True)
    enc_idx.append(df_temp)
for i in range(len(enc_idx)):
    enc_idx[i].set_index('日期', inplace=True)
    print(len(enc_idx[i]))

1293
1272
1316
1263
1263


In [3]:
target = pd.read_csv('00677U.csv')
target = target[['Date', 'Close']]
target['Date'] = pd.to_datetime(target['Date'])
target.set_index('Date', inplace=True)
target.columns = ['00677']

In [4]:
df = target.join(enc_idx)
df.fillna(method='pad', inplace=True)
col_map = ['FB_VIX','S&P 500','美元','道瓊','那斯達克']
df.columns = col_map
df = df[155:]
df.to_csv('df.csv')
df

,FB_VIX,黃金,原油,美元,道瓊,那斯達克
2017-06-30,9.97,1240.70,46.04,30.3530,21349.63,6140.42
2017-07-03,9.97,1240.70,46.04,30.4170,21479.27,6110.06
2017-07-04,10.18,1240.70,46.04,30.4170,21479.27,6110.06
2017-07-05,10.15,1220.40,45.13,30.5110,21478.17,6150.86
2017-07-06,10.15,1222.20,45.52,30.4850,21320.04,6089.46
...,...,...,...,...,...,...
2020-05-13,6.53,1726.70,25.57,29.7880,23247.97,8863.17
2020-05-14,7.17,1741.90,27.73,29.8920,23625.34,8943.72
2020-05-15,6.83,1754.10,29.65,29.8760,23685.42,9014.56
2020-05-18,6.52,1737.10,33.23,29.9540,24597.37,9234.83
